This notebook, by [felipe.alonso@urjc.es](mailto:felipe.alonso@urjc.es)

In this notebook we will analyze and compare different non-parametric methods over the `pima_indian_diabetes` dataset. Specifically, we will learn:

- How to train $K-$NN, DTs, Random Forest, Gradient Boosting and MLP algorithms.
- How to set their hyperparameters (model selection)
- How to evaluate and compare their performance (model evaluation)


In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

# 0. Load the data 

In [ ]:
path = './data/'
filename = 'pima_indian_diabetes.csv'
df = pd.read_csv(path + filename)

# take a look to the data
df.head()

# 1. Model Selection and evaluation on $K-$NN

We will focus first on the **classification task**, using `Outcome` as target

First, we split our data, but we do it wisely in order build a test set as similar as the train set. Since our target variables is *imbalaced* we might want to activate the `stratify` option.

Take a look to the function documentation: [`train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html?highlight=train_test_split#sklearn.model_selection.train_test_split)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df.drop(['Outcome'], axis=1), df['Outcome'], 
    test_size=0.3, 
    random_state=42, 
    stratify=df['Outcome']
)

print('- Train size:', X_train.shape)
print('- Test size:', X_test.shape)

print('\n- Train target distribution: ', y_train.value_counts().values/len(y_train))
print('- Test target distribution:  ',y_test.value_counts().values/len(y_test))

<div class = "alert alert-success">
Run the above cell several times, does the target variable distribution change?
</div>

### Standarize/normalize variables

$K-$NN and MLP require the input data to be standarized

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_train)

X_train_norm = scaler.transform(X_train)
X_test_norm  = scaler.transform(X_test)

## 1.1 Let's set a baseline

Before running our algorithms it is always a good practice to build a baseline model, so you we have a reference of what the results should be.

- In `src.utils` you have implemented some functions to help us calculating and representing different classification metrics. 

- The `analyze_train_test_performance` function provides a comparative summary between training and test metrics.

In [ ]:
from sklearn.linear_model import LogisticRegression
from src.utils import analyze_train_test_performance  

# Logistic regression
lr_model = LogisticRegression().fit(X_train_norm,y_train)

# This a custom function, take a look in src.utils
analyze_train_test_performance(lr_model,X_train_norm,X_test_norm,y_train,y_test)

<div class = "alert alert-success">
Does this model overfit? Justify your answer
</div>

## 1.2 Model selection: *GridSearch* 

We are going to sweep different values of the parameters of each algorithm, to determine its optimal value. In this sweep, we will use a cross-validation strategy, but never the test set!

To do so, we will be using the [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html?highlight=gridsearchcv#sklearn.model_selection.GridSearchCV) function.

In [ ]:
from sklearn.model_selection import GridSearchCV

# These are customized functions: should be in utils.py
def hyper_parameters_search(clf, X, y, param_grid, scorer = 'f1', cv=5):
    
    grid = GridSearchCV(clf, param_grid = param_grid, scoring = scorer, cv = cv)
    grid.fit(X, y)

    print("best mean cross-validation score: {:.3f}".format(grid.best_score_))
    print("best parameters: {}".format(grid.best_params_))
    
    return grid

def plot_cv_scoring(grid, hyper_parameter, scorer = 'f1', plot_errors = False, log=False):
    
    scores = np.array(grid.cv_results_['mean_test_score'])
    std_scores = grid.cv_results_['std_test_score']
        
    params = grid.param_grid[hyper_parameter]
    
    if log:
        params = np.log10(params)
    
    if plot_errors:
        plt.errorbar(params,scores,yerr=std_scores, fmt='o-',ecolor='g')
    else:
        plt.plot(params,scores, 'o-')
    plt.xlabel(hyper_parameter,fontsize=14)
    plt.ylabel(scorer)
    plt.show()

#### Train an algorithm using `GridSearchCV`

We need to define:

- `scoring`: strategy to evaluate the performance of the cross-validated model on the validation sets. [Metrics in sklearn](https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter)

<div class = "alert alert-success">
Take a look to the different metrics in sklearn
</div>

- `param_grid`: dictionary with parameters names (`str`) as keys and lists of parameter settings to try as value. Example:

```python
param_grid = {'n_neighbors': range(1,25)}
```




### $K-$NN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Metric for the scoring
scorer = 'accuracy'  # Other possibilities: accuracy, balanced_accuracy, f1, roc_auc, ....

# param_grid
param_grid = {'n_neighbors': range(1,25)}

# Our customized function
grid_knn = hyper_parameters_search(KNeighborsClassifier(), X_train_norm, y_train, param_grid, scorer = scorer)

# do the plotting
plot_cv_scoring(grid_knn,'n_neighbors',scorer, plot_errors=True)

<div class = "alert alert-success">

Which is the optimal value for $k$? You might want to consider
    
- Different score metrics to guide the CV process<br> 
- Plot the CV errors (`plot_errors = True`) 
</div>

## 1.3 Model evaluation

Once we have selected the model (hyper)-parameters, we evaluate its performance

In [ ]:
knn_model =  KNeighborsClassifier(**grid_knn.best_params_).fit(X_train_norm,y_train)

analyze_train_test_performance(knn_model,X_train_norm,X_test_norm,y_train,y_test)

<div class = "alert alert-success">
Does this model overfit? Is it better than our baseline?
</div>

# 2. Decision trees (DTs)

We will repeat the above process for DTs. In this case, the hyper-parameter is `max_depth`.

## 2.1 Model selection

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Decision trees
param_grid = {'max_depth': range(1,15)}

# your code here
# ...

scorer = 'accuracy'
grid_dt = hyper_parameters_search(
    DecisionTreeClassifier(random_state=0), 
    X_train_norm, y_train, param_grid, scorer = scorer, cv=5)

# do the plotting
plot_cv_scoring(grid_dt,'max_depth',scorer, plot_errors=True)

## 2.2 Model evaluation

In [ ]:
# your code here
# 
# dt_model =  ...

dt_model =  DecisionTreeClassifier(random_state=0, max_depth=2).fit(X_train,y_train)

analyze_train_test_performance(dt_model,X_train,X_test,y_train,y_test)

<div class = "alert alert-success">

- Does this model overfit? 
- Is it better than our baseline?
- What if we change `max_depth`?

</div>

## 2.3 DTs visualization

Trees can be visualized using the [`plot_tree`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.plot_tree.html#sklearn.tree.plot_tree) function

In [ ]:
from sklearn.tree import plot_tree

# set plot dimensions
plt.figure(figsize=(10,6))

feature_names=df.columns.drop('Outcome')

plot_tree(
    dt_model,
    feature_names=feature_names, 
    class_names=['non-diabetic','diabetic'], 
    filled=True
)

plt.show()

## 2.4 Feature importance

In [ ]:
# feature importance
def plot_importances(importances, feat_names):
    
    df_importances = pd.Series(importances, index=feat_names)
    
    plt.figure()
    df_importances.plot.bar()
    plt.ylabel("Feature Importance")
    plt.show()

In [ ]:
# Plot feature importance
plot_importances(dt_model.feature_importances_, feature_names)

# 3. Random Forest (RF)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Random Forest
param_grid = {'max_depth': range(1,5),
              'n_estimators' : [50,100,200,500,1000]}

# 3.1.Model selection ... this might take a while

# your code here
# ...

grid_rf = hyper_parameters_search(
    RandomForestClassifier(random_state=0), 
    X_train_norm, y_train, 
    param_grid, scorer = 'accuracy')

In [ ]:
# 3.2 Evaluation

# your code here
# ...
rf_model =  RandomForestClassifier(random_state=0, **grid_rf.best_params_).fit(X_train_norm,y_train)
analyze_train_test_performance(rf_model,X_train_norm,X_test_norm,y_train,y_test)

In [ ]:
# 3.3 Feature importance

# your code here
# ...
plot_importances(rf_model.feature_importances_, feature_names)

<div class = "alert alert-success">

- Does this model overfit? 
- Is it better than our baseline?
- What are the most important features?

</div>

# 4. Gradient Boosting (trees) (BT)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

param_grid = {
    'n_estimators' : [50,100,200,500, 1000, 2000],
    'learning_rate': [0.1,0.05,0.01, 0.005, 0.001], 
    'max_depth': [1, 2]
} 

# 4.1 Model selection ... this might take a while

# your code here
#  ...

grid_bt = hyper_parameters_search(
    GradientBoostingClassifier(random_state=0), 
    X_train_norm, y_train, param_grid, scorer='accuracy')

In [ ]:
# 4.2 Model evaluation

# your code here
#  ...

bt_model =  GradientBoostingClassifier(random_state=0, **grid_bt.best_params_).fit(X_train_norm,y_train)
analyze_train_test_performance(bt_model,X_train_norm,X_test_norm,y_train,y_test)



In [ ]:
# 4.3 Feature importance

# your code here
# ...
plot_importances(bt_model.feature_importances_, feature_names)

<div class = "alert alert-success">

- Does this model overfit? 
- Is it better than our baseline?
- What are the most important features?

</div>

# 5. Multilayer Perceptron (MLP)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.neural_network import MLPClassifier

param_grid = {
    'alpha' : 10.0 ** -np.arange(1, 7), # following recommendation: https://scikit-learn.org/stable/modules/neural_networks_supervised.html#tips-on-practical-use
    'hidden_layer_sizes': [(5,), (10,), (5,5)] # [(5 neurons, 1 hidden layer), (10 neurons, 1 hidden layer) ,...]
} 

# 5.1 Model selection ...

# your code here
#  ...
param_grid = {
    'alpha' : 10.0 ** -np.arange(1, 7), # following recommendation: https://scikit-learn.org/stable/modules/neural_networks_supervised.html#tips-on-practical-use
    'hidden_layer_sizes': [(5,), (10,), (5,5)] # [(5 neurons, 1 hidden layer), (10 neurons, 1 hidden layer)]
} 

grid_mlp = hyper_parameters_search(
    MLPClassifier(random_state=0), 
    X_train_norm, y_train, param_grid, scorer='accuracy'
)

In [ ]:
# 5.2 Model evaluation

# your code here
#  ...
mlp_model =  MLPClassifier(random_state=0, **grid_mlp.best_params_).fit(X_train_norm,y_train)
analyze_train_test_performance(mlp_model,X_train_norm,X_test_norm,y_train,y_test)

<div class = "alert alert-success">

- Does this model overfit? 
- Is it better than our baseline?
- What are the most important features?

</div>

# 6. Model comparison

If you have done all of the above, just run the following cell

In [ ]:
from sklearn.metrics import roc_curve, auc

modelos = {'LR': lr_model, 'KNN':knn_model,'DT':knn_model, 'RF': rf_model, 'BT': bt_model, 'MLP': mlp_model}

plt.figure(figsize=(7,5))
plt.plot([0, 1], [0, 1], 'k--')
for k,v in modelos.items():
    fpr, tpr,_ = roc_curve(y_test, modelos[k].predict_proba(X_test_norm)[:,1])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label = k + ': %0.2f' % roc_auc)

plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc="lower right")
plt.show()

<div class = "alert alert-success">

Taking into account the above results, in your opinion
    
- What is the best model? Justify your answer 
- And the most important feature(s)? 
- If you want to improve these results, what would you do?

</div>

# 7. Your turn!

<div class = "alert alert-success">

1. Apply non-parametric methods to a regression problem. 
    - Which performance metric would you use?


2. Apply non-parametric methods to your dataset. 

</div> 

# Project Ideas

Here there are some tips that you might want to consider for your project:

1. Select a baseline classifier. Which one did you choose? Why?


2. Compare different machine learning methods. Which one provides you with the best performance? Comment on:
    - ... how you selected the best model (hyperparameters you chose and the performance metric you used to do so)
    - ... model overfitting


3. If you used decision trees or decision trees based algorithms, which are the most important features? Are they coherent with you domain expertise?


In all above, justify your decisions.